# IDS CARGA MASIVA
Este Script tomará el excel de carga masiva, y sustituirá el id previo de cada regla si lo encuentra por el id funcional.

Es necesario rellanar el csv `.\reemplazo.csv` y hacer pull de los repositorios a modificar.  
Ejemplo:  
> old,new,confs  
> GL_MVP_t_re4m_virtual_assistant_message_3.1_000,2882f7df11,RE4M_virtual-assistant-message_MRField.conf  

Automaticamente se traerá los confs los reemplazara y los sustituirá por los confs en el repo.  

In [1]:
import re
import os
import pandas as pd
import shutil
import collections

from pyhocon.config_parser import STR_SUBSTITUTION

from pyhocon import ConfigFactory

In [2]:
confs_directory = 'reemplazo_confs'
reemplazo_file = 'reemplazo.csv'

In [3]:
def parser_conf(conf_content, replace_ids):
    conf_data = ConfigFactory.parse_string(conf_content, resolve=False, unresolved_value=STR_SUBSTITUTION)
    rules = conf_data.get_list('hammurabi.rules')
    for rule in rules:
        prev_id = rule.config.id
        try:
            new_id = replace_ids[prev_id]
            pattern = r'\n\s*id\s*[:|=]\s*"*{}"*\s*\n'.format(re.escape(prev_id))
            replacement = r'\n\t\t\t\tid:"{}"\n'.format(new_id)
            conf_content = re.sub(pattern, replacement, conf_content)
        except Exception as e:
            try:
                new_id = replace_ids[str(prev_id)]
                pattern = r'\n\s*id\s*[:|=]\s*"*{}"*\s*\n'.format(re.escape(str(prev_id)))
                replacement = r'\n\t\t\t\tid:"{}"\n'.format(new_id)
                conf_content = re.sub(pattern, replacement, conf_content)
            except Exception as e:
                print("ERROR" ,e)
    return conf_content

def process_config_files(directory, replace_ids_dict):
    for filename in os.listdir(directory):
        if filename.endswith(".conf"):
            file_path = os.path.join(directory, filename)
            with open(file_path, "r") as file:
                conf_content = file.read()

            new_content = parser_conf(conf_content, replace_ids_dict)

            with open(file_path, "w") as file:
                file.write(new_content)


In [4]:
df = pd.read_csv('reemplazo.csv')
confs = list(df.confs.dropna().unique())
print('- CONFS:')
print(confs)
ids             = df['new'].tolist()
previous_ids    = df['old'].tolist()
print('\n- RULE replaces:')
replace_ids_dict = {prev_id: new_id for new_id, prev_id in zip(ids, previous_ids)}
replace_ids_dict

- CONFS:
['L9CT_prod-assets-oilgas_MRField.conf', 'L9CT_prod-assets-oilgas-annual_MRField.conf', 'EJB6_digital-sales_MRField.conf', 'L9CT_company-metric-year-cntry_MRField.conf', 'L9CT_company-metric-year-cntry_L9CT_esg-ext-data-companies_MMRule.conf']

- RULE replaces:


{'GL_MVP_t_ejb6_digital_sales_3.5_002': '28ada4d9a3',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.1_000': '28ada4da8d',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.1_001': '28ada4db5b',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.2_000': '28ada4dc26',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.2_001': '28ada4dced',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.5_000': '28ada4ddaf',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.5_001': '28ada4de86',
 'GL_MVP_t_l9ct_company_metric_year_cntry_3.5_002': '28ada4df4d',
 'GL_MVP_t_l9ct_prod_assets_oilgas_3.5_000': '28ada4e021',
 'GL_MVP_t_l9ct_prod_assets_oilgas_3.5_001': '28ada4e0ef',
 'GL_MVP_t_l9ct_prod_assets_oilgas_3.5_002': '28ada4e1cd',
 'GL_MVP_t_l9ct_prod_assets_oilgas_annual_3.5_000': '28ada4e2a3',
 'GL_MVP_t_l9ct_prod_assets_oilgas_annual_3.5_001': '28ada4e375',
 'GL_STR_t_l9ct_company_metric_year_cntry_4.2_000': '28ada4e43c',
 'GL_STR_t_l9ct_company_metric_year_cntry_4.3_00N': '28ada4e4f9',
 'GL_STR_t_l9ct_company_metric_year_cntry_5

In [5]:
len(replace_ids_dict)

16

In [6]:
print('hay ids duplicados?')
print([item for item, count in collections.Counter(ids).items() if count > 1])

hay ids duplicados?
[]


In [7]:
# Reseteamos directorio y nos traemos confs
print(confs)
shutil.rmtree('./reemplazo_confs/')
os.makedirs('./reemplazo_confs/')
for c in confs:
    uuaa = c[:4].lower()
    dst = './reemplazo_confs/'+c
    src = 'C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/{0}/glgldh360gloqlt/hammurabi/confs/master/{0}/{1}'.format(uuaa,c)
    print(c,src)
    shutil.copyfile(src, dst)

['L9CT_prod-assets-oilgas_MRField.conf', 'L9CT_prod-assets-oilgas-annual_MRField.conf', 'EJB6_digital-sales_MRField.conf', 'L9CT_company-metric-year-cntry_MRField.conf', 'L9CT_company-metric-year-cntry_L9CT_esg-ext-data-companies_MMRule.conf']
L9CT_prod-assets-oilgas_MRField.conf C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/l9ct/glgldh360gloqlt/hammurabi/confs/master/l9ct/L9CT_prod-assets-oilgas_MRField.conf
L9CT_prod-assets-oilgas-annual_MRField.conf C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/l9ct/glgldh360gloqlt/hammurabi/confs/master/l9ct/L9CT_prod-assets-oilgas-annual_MRField.conf
EJB6_digital-sales_MRField.conf C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/ejb6/glgldh360gloqlt/hammurabi/confs/master/ejb6/EJB6_digital-sales_MRField.conf
L9CT_company-metric-year-cntry_MRField.conf C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/l9ct/glgldh360gloqlt/hammurabi/confs/master/l9ct/L9CT_company-metric-year-cntry

In [8]:
# realizamos el reemplazo de ids
process_config_files(confs_directory,replace_ids_dict)


ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_000'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_001'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_002'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_003'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_004'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_005'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.1_006'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_000'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_001'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_002'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_003'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_004'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_005'
ERROR 'GL_MVP_t_ejb6_digital_sales_3.2_006'
ERROR 'GL_DOM_t_ejb6_digital_sales_3.5_000'
ERROR '2801aec1d5'
ERROR '2801aec24e'
ERROR '289cc78dc2'
ERROR '289cc78ebe'
ERROR '289cc78fc9'
ERROR '289cc790bb'
ERROR '289cc791a7'
ERROR 'GL_STR_t_ejb6_digital_sales_4.2_000'
ERROR 'GL_MVP_t_l9ct_prod_assets_oilgas_annual_3.1_000'
ERROR 'GL_MVP_t_l9ct_prod_assets_oilgas_annual_3.1_001'
ERROR 'GL_MVP_t_l9ct_prod_assets_oilgas_annual_3.2_

In [9]:
# una vez realizado el cambio y revisado;), llevamos los confs a los repos:
###############################################################################
uas = []
for c in confs:
    uuaa = c[:4].lower()
    src = './reemplazo_confs/'+c
    dst = 'C:/Users/O005820/Documents/repositorios/GDH_c360_hammurabi_UUAAs/{0}/glgldh360gloqlt/hammurabi/confs/master/{0}/{1}'.format(uuaa,c)
    print(c,src)
    shutil.copyfile(src, dst)
    uas.append(uuaa)


L9CT_prod-assets-oilgas_MRField.conf ./reemplazo_confs/L9CT_prod-assets-oilgas_MRField.conf
L9CT_prod-assets-oilgas-annual_MRField.conf ./reemplazo_confs/L9CT_prod-assets-oilgas-annual_MRField.conf
EJB6_digital-sales_MRField.conf ./reemplazo_confs/EJB6_digital-sales_MRField.conf
L9CT_company-metric-year-cntry_MRField.conf ./reemplazo_confs/L9CT_company-metric-year-cntry_MRField.conf
L9CT_company-metric-year-cntry_L9CT_esg-ext-data-companies_MMRule.conf ./reemplazo_confs/L9CT_company-metric-year-cntry_L9CT_esg-ext-data-companies_MMRule.conf


In [10]:
print('UUAAs a mergear:')
print([item for item, count in collections.Counter(uas).items()])

UUAAs a mergear:
['l9ct', 'ejb6']
